In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import torch
from torch.utils.data import DataLoader
import random
import torch.nn.functional as F
from torchmetrics.functional.image.ssim import (
    structural_similarity_index_measure,
)
from torchmetrics.functional.image.psnr import peak_signal_noise_ratio

In [2]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
random.seed(42)
g = torch.Generator()
g.manual_seed(42)

In [3]:
def resize_encoder(sample):
    sample = sample.unsqueeze(0)
    sample = F.interpolate(
        sample, size=(144, 72), mode="bicubic", align_corners=False
    )
    return sample.squeeze(0)


def random_mask(sample, mask_prob_low=0.7, mask_prob_high=0.7):
    if mask_prob_low == mask_prob_high:
        mask_prob = mask_prob_low
    else:
        mask_prob = random.uniform(mask_prob_low, mask_prob_high)
    random_tensor = torch.rand(sample.shape, device=sample.device)
    mask = (random_tensor > mask_prob).float()
    masked_image = sample * mask
    return masked_image


class WeatherBenchDataset(Dataset):
    def __init__(self, data, mask_prob_low=0.7, mask_prob_high=0.7):
        self.data = data
        self.mask_prob_low = mask_prob_low
        self.mask_prob_high = mask_prob_high

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        X = self.data[idx]
        X_enc = resize_encoder(X)
        X_masked = random_mask(
            X_enc,
            mask_prob_low=self.mask_prob_low,
            mask_prob_high=self.mask_prob_high,
        )
        return X, X_masked, X_enc

In [4]:
# RRMSE
def RRMSE(preds, target):
    mse = torch.mean(((preds - target) ** 2))
    rmse = torch.sqrt(mse)
    rrmse = rmse / (target.mean()) * 100
    return rrmse

In [5]:
# See: https://arxiv.org/pdf/2407.15171
def gaussian_density(z_g, Z, sigma):
    z_g = z_g.float()
    Z = Z.float()

    if z_g.dim() == 1:
        z_g = z_g.unsqueeze(0)
        squeeze_out = True
    else:
        squeeze_out = False

    # Compute squared Euclidean distances:
    #   diff: (B, N, d)
    #   sq_dist: (B, N)
    diff = Z.unsqueeze(0) - z_g.unsqueeze(1)
    sq_dist = (diff**2).sum(dim=-1)

    weights = torch.exp(-sq_dist / (2 * sigma**2))
    D = weights.mean(dim=-1)

    return D.squeeze(0) if squeeze_out else D

In [6]:
# Realism Score
# https://arxiv.org/pdf/1904.06991
def realism_score(real_feats, gen_feats, k):
    N_r, D = real_feats.shape
    # Distance for nearest neighbours
    dist_rr = torch.cdist(real_feats, real_feats)
    # ignore self‐distance
    idx = torch.arange(N_r, device=real_feats.device)
    dist_rr[idx, idx] = float("inf")
    r_k, _ = dist_rr.kthvalue(k, dim=1)
    dist_gr = torch.cdist(gen_feats, real_feats)  # (N_g, N_r)
    # Realism matrix
    ratios = r_k.unsqueeze(0) / dist_gr  # (N_g, N_r)
    # Max over reals
    realism, _ = ratios.max(dim=1)
    return realism

In [7]:
def plot_image(xh, x_full):
    num_channels = xh.shape[0]
    mode_labels = [
        "2m_temperature",
        "u_component_of_wind",
        "v_component_of_wind",
        "geopotential",
        "specific_humidity",
    ]
    
    fig = plt.figure(figsize=(num_channels * 2.2, 5))
    gs = gridspec.GridSpec(2, num_channels, wspace=0.02, hspace=0.2)

    for i in range(num_channels):
        ax = fig.add_subplot(gs[0, i])
        ax.imshow(x_full[i], cmap="coolwarm")
        ax.set_title(mode_labels[i], fontsize=10)
        ax.axis("off")

        ax = fig.add_subplot(gs[1, i])
        ax.imshow(xh[i], cmap="coolwarm")
        ax.set_title(mode_labels[i], fontsize=10)
        ax.axis("off")

    plt.show()
    plt.close()


In [8]:
latent_model = torch.load(
    "conditional_latent_model.pth", weights_only=False, map_location=DEVICE
)
model = torch.load(
    "simclr_decoder_freeze.pth", weights_only=False, map_location=DEVICE
)
latent_model.eval()
model.eval()

encoder_model = model.model.encoder
decoder_model = model.decoder

data = torch.load("/vol/bitbucket/nb324/ERA5_64x32_daily_850.pt")
n_samples = data.shape[0]
n_train = int(n_samples * 0.6)
n_valid = int(n_samples * 0.2)
data = data[n_train + n_valid :]


BATCH_SIZE = 128
NUM_TIMES_TO_SAMPLE = 10
n_samples = data.shape[0]

n_train = int(n_samples * 0.6)
n_valid = int(n_samples * 0.2)

train_data = data[:n_train]
valid_data = data[n_train : n_train + n_valid]
test_data = data[n_train + n_valid :]

mean = train_data.mean(dim=(0, 2, 3), keepdim=True)
std = train_data.std(dim=(0, 2, 3), keepdim=True)

test_data = (test_data - mean) / std

In [9]:
test_dataset = WeatherBenchDataset(
    data=test_data, mask_prob_low=0.7, mask_prob_high=0.7
)
testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [10]:
# Encode the data

encoded_data = []
full_data = []
with torch.no_grad():
    for data in testloader:
        x_f_batch = data[0].to(DEVICE)
        x_mask = data[1].to(DEVICE)
        x_mask_enc, _ = encoder_model(x_mask)
        encoded_data.append(x_mask_enc)
        full_data.append(x_f_batch)

full_data = torch.cat(full_data, dim=0)
encoded_data = torch.cat(encoded_data, dim=0)

In [11]:
# Generate some samples
batch = random.choice(list(testloader))
condition = batch[1].to(DEVICE)
condition, _ = encoder_model(condition)

# Take the average generated image
zh_samples = []
zh_sum = torch.zeros(condition.shape[0], 1000).to(DEVICE)
for i in range(NUM_TIMES_TO_SAMPLE):
    with torch.no_grad():
        zh = latent_model.sample(condition.shape[0], 1000, condition)
    zh_samples.append(zh)
    zh_sum += zh

zh_samples = torch.cat(zh_samples, dim=0)
zh_mean = zh_sum / NUM_TIMES_TO_SAMPLE
print(f"Standard Deviation in Samples: {zh_samples.std():.4f}")

Standard Deviation in Samples: 2.5985


In [12]:
# Compute gaussian density score
gaussian_density_score = gaussian_density(
    z_g=zh_mean, Z=encoded_data, sigma=20.0
)
gaussian_density_score = gaussian_density_score.mean()
print(f"Gaussian Density Score: {gaussian_density_score.item()}")

Gaussian Density Score: 0.466459184885025


In [13]:
# Realism Score
realism_score_val = realism_score(
    real_feats=encoded_data, gen_feats=zh_mean, k=5
)
realism_score_val = realism_score_val.mean()
print(f"Realism Score: {realism_score_val.item()}")

Realism Score: 0.18274450302124023


In [14]:
zh_mean = zh_mean.to(DEVICE)
with torch.no_grad():
    xh = decoder_model(zh_mean)
x_f = batch[0].cpu()
xh = xh.cpu()

In [15]:
x_f = batch[0].cpu()
mse_l = F.mse_loss(xh, x_f)

decoded_data_ = xh * std + mean
output_data_ = x_f * std + mean

rrmse_loss = RRMSE(decoded_data_, output_data_)

x_min = train_data.amin(dim=(0, 2, 3), keepdim=True)
x_max = train_data.amax(dim=(0, 2, 3), keepdim=True)

decoded_data_ = (decoded_data_ - x_min) / (x_max - x_min + 1e-8)
output_data_ = (output_data_ - x_min) / (x_max - x_min + 1e-8)

ssim = structural_similarity_index_measure(
    decoded_data_, output_data_, data_range=1.0
)
psnr = peak_signal_noise_ratio(decoded_data_, output_data_, data_range=1.0)

print(f"MSE Batch Loss: {mse_l:.4f}")
print(f"RRMSE Batch Loss: {rrmse_loss:.4f}")
print(f"SSIM Batch: {ssim}")
print(f"PSNR Batch: {psnr}")

MSE Batch Loss: 2.3320
RRMSE Batch Loss: 29.3769
SSIM Batch: 0.36474111676216125
PSNR Batch: 16.004318237304688


In [16]:
xh = xh[0]
x_f = x_f[0]

In [17]:
mse_l = F.mse_loss(xh, x_f)
decoded_data_ = xh * std + mean
output_data_ = x_f * std + mean
rrmse_loss = RRMSE(decoded_data_, output_data_)
decoded_data_ = (decoded_data_ - x_min) / (x_max - x_min + 1e-8)
output_data_ = (output_data_ - x_min) / (x_max - x_min + 1e-8)
ssim = structural_similarity_index_measure(
    decoded_data_, output_data_, data_range=1.0
)
psnr = peak_signal_noise_ratio(decoded_data_, output_data_, data_range=1.0)
xh_np = xh.numpy()
x_f_np = x_f.numpy()
plot_image(xh_np, x_f_np)
print(f"MSE Loss: {mse_l:.4f}")
print(f"RRMSE Loss: {rrmse_loss:.4f}")
print(f"SSIM Batch: {ssim}")
print(f"PSNR Batch: {psnr}")

MSE Loss: 1.9372
RRMSE Loss: 30.1427
SSIM Batch: 0.362918883562088
PSNR Batch: 16.60951805114746
